In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from networks import *
from menet.menets_rev import MeNets

In [ ]:
#@title Load preprocessed & subsampled data (LOOCV)

ids =                    np.load('../mpii_dataset/within_ids.npy')
images = torch.as_tensor(np.load('../mpii_dataset/within_images.npy'), dtype=torch.float)
hps =    torch.as_tensor(np.load('../mpii_dataset/within_2d_hps.npy'), dtype=torch.float)
gazes =  torch.as_tensor(np.load('../mpii_dataset/within_2d_gazes.npy'), dtype=torch.float)

In [ ]:
device = torch.device('cuda:0')
seed = 10

experiment_name = 'mpii_within'
res_list = []

In [ ]:
for fold in range(10) : 

    train_ids = np.concatenate([ids[:fold], ids[(fold+1):]]).reshape(-1)
    train_images = torch.cat([images[:fold], images[(fold+1):]]).reshape(-1,36,60)
    train_hps = torch.cat([hps[:fold], hps[(fold+1):]]).reshape(-1,2)
    train_gazes = torch.cat([gazes[:fold], gazes[(fold+1):]]).reshape(-1,2)

    test_ids = ids[fold]
    test_images = images[fold]
    test_hps = hps[fold]
    test_gazes = gazes[fold]

    res_list.append(MeNets(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        ResNet.ResNet, max_iter=1, patience=1,
        test_unseen=False, result_path = '.', save=True,
        device=device, SEED=seed + fold, experiment_name = f'{experiment_name}_{fold}'
    ))

